In [ ]:
%pip install numpy plotly matplotlib nbformat pandas

In [4]:
import numpy as np
import plotly.express as px

In [12]:
origin = np.array([0, 0, 1])


def sierpinksy():
    scalar = np.array(
        [
            [0.5, 0, 0],
            [0, 0.5, 0],
            [
                0,
                0,
                1,
            ],
        ]
    )

    def translation(x, y):
        return np.array([[1, 0, x], [0, 1, y], [0, 0, 1]])

    return [
        np.matmul(translation(0, 0.36), scalar),
        np.matmul(translation(-0.5, -0.5), scalar),
        np.matmul(translation(0.5, -0.5), scalar),
    ]


def barnsley_fern():
    def make_matrix(f, t):
        return np.array([[f[0][0], f[0][1], t[0]], [f[1][0], f[1][1], t[1]], [0, 0, 1]])

    f1 = [[0, 0], [0, 16]]
    f2 = [[85, 4], [-4, 85]]
    f3 = [[2, -26], [23, 22]]
    f4 = [[-15, 28], [26, 24]]
    fs = [f1, f2, f3, f4]

    t1 = [0, 0]
    t2 = [0, 16]
    t3 = [0, 16]
    t4 = [0, 44]
    ts = [t1, t2, t3, t4]

    return list(map(lambda z: make_matrix(*z), zip(fs, ts)))


As = barnsley_fern()
for A in As:
    display(A)

array([[ 0,  0,  0],
       [ 0, 16,  0],
       [ 0,  0,  1]])

array([[85,  4,  0],
       [-4, 85, 16],
       [ 0,  0,  1]])

array([[  2, -26,   0],
       [ 23,  22,  16],
       [  0,   0,   1]])

array([[-15,  28,   0],
       [ 26,  24,  44],
       [  0,   0,   1]])

In [13]:
iters = 400
N = 2560
rng = np.random.default_rng(10)
Xs = []
Ys = []
for i in range(N):
    p = np.copy(origin)
    for j in range(iters):
        A = As[int(np.floor(rng.random() * len(As)))]
        p = np.matmul(A, p)
    Xs.append(p[0])
    Ys.append(p[1])
AABB_prob = np.array([[np.min(Xs), np.min(Ys)], [np.max(Xs), np.max(Ys)]])
px.scatter(x=Xs, y=Ys).show()
AABB_prob

array([[-9220745094228642976, -9216975579036841968],
       [ 9208432000488687104,  9220885102947763712]])

In [14]:
values = [origin]
for j in range(min(iters, 8)):
    new_values = []
    for v in values:
        for A in As:
            new_values.append(np.matmul(A, v))
    values = new_values
values = np.array(values)
print(len(values))

AABB_ref = np.array([[np.min(values[:, 0]), np.min(values[:, 1])], [np.max(values[:, 0]), np.max(values[:, 1])]])
AABB_ref

65536


array([[-409649739656504,  -72870229714852],
       [ 565538575473072, 1351088442557420]])

In [15]:
from math import log

mal = origin
mil = origin
ds = []
for i in range(iters):
    mi = np.array([np.inf, np.inf])
    ma = -np.copy(mi)
    for A in As:
        Amal = np.matmul(A, mal)
        Amil = np.matmul(A, mil)
        mi[0] = min(mi[0], Amil[0])
        mi[1] = min(mi[1], Amil[1])
        ma[0] = max(ma[0], Amal[0])
        ma[1] = max(ma[1], Amal[1])
    ma = np.array([*ma, 1]) 
    mi = np.array([*mi, 1]) 
    d = 100*(np.linalg.norm(mal - ma) / np.linalg.norm(AABB_ref[1]) + np.linalg.norm(mil - mi) / np.linalg.norm(AABB_ref[0]))
    ds.append([i, d])
    mal = ma
    mil = mi

aabb = np.array([mil, mal]).T
ds = np.array(ds)
px.scatter(x=Xs, y=Ys).add_scatter(x=aabb[0], y=aabb[1]).show()
px.scatter(x=ds[:, 0], y=ds[:, 1])

/tmp/ipykernel_5893/2275572980.py:10: RuntimeWarning:

overflow encountered in matmul

/tmp/ipykernel_5893/2275572980.py:10: RuntimeWarning:

invalid value encountered in matmul

/tmp/ipykernel_5893/2275572980.py:18: RuntimeWarning:

invalid value encountered in subtract

